In [150]:
import csv
import pandas as pd
import mysql.connector

mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd="NYg1@nts",
  database="NRL_data"
)
mycursor = mydb.cursor()

In [151]:
csv_name = ('../../scraping/scraper_py/csv_files/2018_data.csv')

open_file = open(csv_name)
read_file = list(csv.reader(open_file))

In [152]:
stat_df = pd.read_csv('../../scraping/scraper_py/csv_files/2018_data.csv', index_col='index').drop(columns='Unnamed: 0')
stat_df.head()

,player_id,team_id,match_id,number,position,minutes_played,points,tries,conversions,conversion_attempts,...,kicked_dead,errors,handling_errors,one_on_ones_lost,penalties,on_report,sin_bins,send_offs,stint_one,stint_two
index,,,,,,,,,,,,,,,,,,,,,
Billy_Slater_7,568.0,7,462,1,Fullback,80:00,0,0,0,0,...,0,1,1,0,0,0,0,0,80:00,0
Suliasi_Vunivalu_7,185.0,7,462,2,Winger,80:00,8,2,0,0,...,0,2,1,0,0,0,0,0,80:00,0
Cheyse_Blair_7,623.0,7,462,3,Centre,80:00,8,2,0,0,...,0,0,0,0,0,0,0,0,80:00,0
Curtis_Scott_7,177.0,7,462,4,Centre,80:00,4,1,0,0,...,0,0,0,0,1,0,0,0,80:00,0
Josh_Addo-carr_7,179.0,7,462,5,Winger,80:00,0,0,0,0,...,0,3,3,0,1,0,0,0,80:00,0


In [153]:
def find_position_id(name):
    find_position_query = 'SELECT id FROM Positions WHERE position_name = %s;'
    mycursor.execute(find_position_query, (name,))
    return mycursor.fetchone()[0]

In [154]:
for column in ['conversion_percentage', 'tackle_percentage']:
    stat_df[column] = stat_df[column].str.replace('%', '').astype('float') / 100
    stat_df[column] = stat_df[column].round(3)
stat_df['average_play_the_ball_seconds'] = stat_df['average_play_the_ball_seconds'].str.replace('s', '').astype('float')
stat_df['position_id'] = stat_df['position'].apply(find_position_id)
stat_df.head()

,player_id,team_id,match_id,number,position,minutes_played,points,tries,conversions,conversion_attempts,...,errors,handling_errors,one_on_ones_lost,penalties,on_report,sin_bins,send_offs,stint_one,stint_two,position_id
index,,,,,,,,,,,,,,,,,,,,,
Billy_Slater_7,568.0,7,462,1,Fullback,80:00,0,0,0,0,...,1,1,0,0,0,0,0,80:00,0,1
Suliasi_Vunivalu_7,185.0,7,462,2,Winger,80:00,8,2,0,0,...,2,1,0,0,0,0,0,80:00,0,2
Cheyse_Blair_7,623.0,7,462,3,Centre,80:00,8,2,0,0,...,0,0,0,0,0,0,0,80:00,0,3
Curtis_Scott_7,177.0,7,462,4,Centre,80:00,4,1,0,0,...,0,0,0,1,0,0,0,80:00,0,3
Josh_Addo-carr_7,179.0,7,462,5,Winger,80:00,0,0,0,0,...,3,3,0,1,0,0,0,80:00,0,2


In [155]:
def convert_time_columns_to_numeric(x):
    x = x.split(':')
    minutes = x[0]
    try:
        seconds = x[1]
    except:
        seconds = 0
    return round(((int(minutes) * 60) + int(seconds)) / 60, 2)
    
for column in ['minutes_played', 'stint_one', 'stint_two']:
    stat_df[column] = stat_df[column].apply(lambda x: convert_time_columns_to_numeric(x))

In [158]:
stat_df['player_id'] = stat_df['player_id'].astype('int')

In [159]:
stat_df.head()

,player_id,team_id,match_id,number,position,minutes_played,points,tries,conversions,conversion_attempts,...,errors,handling_errors,one_on_ones_lost,penalties,on_report,sin_bins,send_offs,stint_one,stint_two,position_id
index,,,,,,,,,,,,,,,,,,,,,
Billy_Slater_7,568,7,462,1,Fullback,80.0,0,0,0,0,...,1,1,0,0,0,0,0,80.0,0.0,1
Suliasi_Vunivalu_7,185,7,462,2,Winger,80.0,8,2,0,0,...,2,1,0,0,0,0,0,80.0,0.0,2
Cheyse_Blair_7,623,7,462,3,Centre,80.0,8,2,0,0,...,0,0,0,0,0,0,0,80.0,0.0,3
Curtis_Scott_7,177,7,462,4,Centre,80.0,4,1,0,0,...,0,0,0,1,0,0,0,80.0,0.0,3
Josh_Addo-carr_7,179,7,462,5,Winger,80.0,0,0,0,0,...,3,3,0,1,0,0,0,80.0,0.0,2


In [ ]:
insert_query = query = '''INSERT INTO PlayerMatchStats (match_id, player_id, team_id, position_id, minutes_played,
    points, tries, conversions, conversion_attempts, penalty_goals,
    conversion_percentage, field_goals, total_runs, total_run_metres, kick_return_metres,
    post_contact_metres, line_breaks, line_break_assists, try_assists, line_engaged_runs,
    tackle_breaks, hit_ups, play_the_ball, average_play_ball_seconds, dummy_half_runs,
    dummy_half_run_metres, steals, offloads, dummy_passes, passes,
    receipts, pass_to_run_ratio, tackle_percentage, tackles_made, tackles_missed, ineffective_tackles,
    intercepts, kicks_defused, kicks, kicking_metres, forced_drop_outs,
    bomb_kicks, grubbers, fourty_twenty, cross_field_kicks, kicked_dead,
    errors, handling_errors, one_on_ones_lost, penalties, on_report,
    sin_bins, send_offs, stint_one, stint_two)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
    %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
    %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
    '''

In [163]:
for row in stat_df.iterrows():
    r = row[1]
    insert_tuple = (r['match_id'],
r['player_id'],
r['team_id'],
r['position_id'],
r['minutes_played'],
r['points'],
r['tries'],
r['conversions'],
r['conversion_attempts'],
r['penalty_goals'],
r['conversion_percentage'],
r['field_goals'],
r['total_runs'],
r['total_run_metres'],
r['kick_return_metres'],
r['post_contact_metres'],
r['line_breaks'],
r['line_break_assists'],
r['try_assists'],
r['line_engaged_runs'],
r['tackle_breaks'],
r['hit_ups'],
r['play_the_ball'],
r['average_play_ball_seconds'],
r['dummy_half_runs'],
r['dummy_half_run_metres'],
r['steals'],
r['offloads'],
r['dummy_passes'],
r['passes'],
r['receipts'],
r['pass_to_run_ratio'],
r['tackle_percentage'],
r['tackles_made'],
r['tackles_missed'],
r['ineffective_tackles'],
r['intercepts'],
r['kicks_defused'],
r['kicks'],
r['kicking_metres'],
r['forced_drop_outs'],
r['bomb_kicks'],
r['grubbers'],
r['fourty_twenty'],
r['cross_field_kicks'],
r['kicked_dead'],
r['errors'],
r['handling_errors'],
r['one_on_ones_lost'],
r['penalties'],
r['on_report'],
r['sin_bins'],
r['send_offs'],
r['stint_one'],
r['stint_two'])
    print(r)

player_id                             568
team_id                                 7
match_id                              462
number                                  1
position                         Fullback
minutes_played                         80
points                                  0
tries                                   0
conversions                             0
conversion_attempts                     0
penalty_goals                           0
conversion_percentage                   0
field_goals                             0
fantasy_points                         28
total_runs                             13
total_run_metres                      105
kick_return_metres                     50
post_contact_metres                    38
line_breaks                             1
line_break_assists                      1
try_assists                             1
line_engaged_runs                       0
tackle_breaks                           2
hit_ups                           

Name: Christian_Welch_7, dtype: object
player_id                                158
team_id                                    7
match_id                                 470
number                                    16
position                         Interchange
minutes_played                            16
points                                     0
tries                                      0
conversions                                0
conversion_attempts                        0
penalty_goals                              0
conversion_percentage                      0
field_goals                                0
fantasy_points                            16
total_runs                                 5
total_run_metres                          48
kick_return_metres                         0
post_contact_metres                       16
line_breaks                                0
line_break_assists                         0
try_assists                                0
line_engaged_run